<a href="https://colab.research.google.com/github/shivam-pandey-15/Generative-Models/blob/master/Tensorflow%20v.2/Conditional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#To run in colab
try:
  %tensorflow_version 2.x
except Exception:
  pass


TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from IPython import display
import time
import random

In [0]:
(train,label) , (test,_)  = tf.keras.datasets.mnist.load_data()

In [0]:
train_image = train.reshape(60000,28*28)/255

In [0]:
EPOCH = 1000
bs = 128
buffer = 60000

In [6]:

label = tf.one_hot(label,10)
tf.cast(train,tf.float64)
tf.cast(label,tf.float64)

<tf.Tensor: id=7, shape=(60000, 10), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])>

In [0]:
train_image = tf.data.Dataset.from_tensor_slices((train_image,label)).shuffle(buffer).batch(128,True)


In [0]:
class CONDITIONAL_GAN(tf.keras.Model):
  
  def __init__(self):
    super(GAN,self).__init__()
    
    
    #Binary Cross Entropy Loss
    
    
    #Generator intitialization
    
    #Model
    self.g_model = tf.keras.Sequential()
    self.g_model.add(layers.Dense(256,activation='relu',input_shape=(110,)))
    self.g_model.add(layers.Dropout(0.2))
    self.g_model.add(layers.Dense(256,activation='relu'))
    self.g_model.add(layers.Dropout(0.2))
    self.g_model.add(layers.Dense(784,activation='tanh'))
    
    #Optimizer
    self.g_optimizer = tf.keras.optimizers.Adam(1e-4)
    
    #Discriminator intitialization
    
    #Model
    self.d_model = tf.keras.Sequential()
    self.d_model.add(layers.Dense(256,activation='relu',input_shape=(794,)))
    self.d_model.add(layers.Dropout(0.2))
    self.d_model.add(layers.Dense(1,activation='sigmoid'))
    
    #Optimizer
    self.d_optimizer = tf.keras.optimizers.Adam(1e-4)
    
  def forward(self,x,label,batch =bs):
    z = tf.random.normal([batch, 100])
    z = tf.cast(z,tf.float64)
    
    z  = tf.concat([z,label],1)
    
    generator_output = self.g_model(z)
    
    generator_output = tf.cast(generator_output,tf.float64)
    generator_output =tf.concat([generator_output,label],1)
    self.fake_output = self.d_model(generator_output)
    self.real_output = self.d_model(x)
    
    
  def loss(self,x):
    
    # Define Cross Entropy
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    # GEnerator Loss
    self.g_loss = cross_entropy(tf.ones_like(self.fake_output), self.fake_output)
    
    
    # Discriminator Loss of real and fake
    d_fake = cross_entropy(tf.zeros_like(self.fake_output), self.fake_output)
    d_real = cross_entropy(tf.ones_like(self.real_output), self.real_output)
    
    self.d_loss = d_fake+d_real
    
    return self.g_loss,self.d_loss
  
  
  @tf.function
  def train(self, x,label):
      
      #Taking step for mini-batch 
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            self.forward(x,label)
            g_loss , d_loss = self.loss(x)
        
      g_gradients = gen_tape.gradient(g_loss, self.g_model.trainable_variables)
      d_gradients = disc_tape.gradient(d_loss, self.d_model.trainable_variables)
      
      self.g_optimizer.apply_gradients(zip(g_gradients,self.g_model.trainable_variables))
      self.d_optimizer.apply_gradients(zip(d_gradients,self.d_model.trainable_variables))
      

     

  

In [0]:
def generate_and_save_images(model, epoch):
  a = random.randint(0,9)
  print(a)
  value = tf.convert_to_tensor([a]*16)
  test_input = tf.random.normal([16, 100])
  predict = tf.one_hot(value,10)
  test_input = tf.concat([test_input,predict],1)
  predictions = model(test_input, training=False).numpy().reshape(-1,28,28,1)
 
  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      a = predictions[i, :, :, 0] * 127.5  
      a[a<10]=0
      plt.imshow(a, cmap='gray')
      plt.axis('off')
  if epoch%10==0:
      plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
      pass
  plt.show()

In [0]:
gan = CONDITIONAL_GAN()

In [0]:
for epoch in range(0,EPOCH+1):
  start = time.time()
  for  x,l in train_image:
    l =tf.cast(l,tf.float64)
    x = tf.concat([x,l],1)
    
    cgan.train(x,l)
  display.clear_output(wait=True)
  generate_and_save_images(gan.g_model,
                             epoch )
    
  print ('Time for epoch {} is {} sec'.format(epoch , time.time()-start))
    
    
    
  